In [1]:
from azure.cognitiveservices.vision.computervision import ComputerVisionClient
from msrest.authentication import CognitiveServicesCredentials

subscription_key = "baae73485243441c8b5295024b15f809"
endpoint = "https://smart-gate-vision.cognitiveservices.azure.com/"

computervision_client = ComputerVisionClient(endpoint, CognitiveServicesCredentials(subscription_key))

remote_image_url = "./motions/event_2024-09-22_07-14-10.png"

read_image = open(remote_image_url, "rb")

vision_response = computervision_client.detect_objects_in_stream(read_image)

for object in vision_response.objects:
    print(object.object_property, object.confidence)

mammal 0.893
